In [120]:
from wms.consulta_estoque import estoque
import sap

sessions = sap.listar_sessoes()
session = sap.criar_sessao(sessions)
materiais = estoque(session, sessions, "4600041302")

Consultando Estoque de Materiais
Planilha de estoque gerada com sucesso.
Encerrando Sessão.
Fechando Arquivo Excel.



In [28]:
lacre = materiais[materiais['Material'] == '50000108']
hidro = materiais[materiais['Material'] == '50000530']
hidro = hidro.query('`Utilização livre` > 1')
if not lacre.empty and not lacre.empty:
    print("ok")

ok


In [46]:
import pandas as pd
num_material_linhas = tb_materiais.RowCount
n_material = 0
ultima_linha_material = num_material_linhas
lista_data = []
for n_material in range(num_material_linhas):
    # Pega valor da célula 0
    sap_material = tb_materiais.GetCellValue(
        n_material, "MATERIAL")
    sap_etapa_material = tb_materiais.GetCellValue(
        n_material, "ETAPA")
    sap_desc_material = tb_materiais.GetCellValue(
        n_material, "DESC_MAT")
    sap_qtde_material = tb_materiais.GetCellValue(
        n_material, "QUANT")
    data = {'Etapa': sap_etapa_material,
            'Material': sap_material,
            'Descrição': sap_desc_material,
            'Quantidade': sap_qtde_material}
    lista_data.append(data)
df = pd.DataFrame(lista_data)
df['Quantidade'] = df['Quantidade'].replace(',', '.', regex=True).astype(float)
df

,Etapa,Material,Descrição,Quantidade
0,0010,50000530,HIDROM VOL DN20 PRE.EQ QN 1.5 M3/H - ASI,1.0
1,0010,50001070,LACRE ANTIFRAUDE P HIDRÔMETRO COMPR 01 M,1.0


In [56]:
if df['Material'].isin(materiais['Material']).any():
    print("foi")

# Alternativa 2
for material in df['Material']:
    if material in materiais['Material'].values:
        print(f"O material {material} está presente.")
        


foi
O material 50000530 está presente.


In [104]:
# Usar o Rodapé para resolver os materiais faltantes.
import re
import sap
connection = sap.listar_conexoes()
session = connection.Children(1)
rodape = session.findById("wnd[0]/sbar").Text # Rodapé
rodape = rodape.lower()
padrao = r"material (\d+)"
correspondencias = re.search(padrao, rodape)
if correspondencias:
    # Group 1 retira string 'material'
    codigo_material = correspondencias.group(1)
    print(codigo_material)

50000263


In [9]:
# Monitor
from datetime import datetime, timedelta
import sap
sessoes = sap.listar_sessoes()
def monitor204():
    data = datetime.today().date().strftime('%d/%m/%Y')
    data = str(data)
    data = data.replace("/", ".")
    hora_inicio = datetime.now() - timedelta(hours=1)
    hora_inicio = hora_inicio.time()
    hora_inicio = hora_inicio.strftime('%H:%M:%S')
    hora_fim = datetime.now().strftime('%H:%M:%S')
    
    session_monitor = sap.criar_sessao(sessoes)
    session_monitor.StartTransaction("ZSBMM204")
    session_monitor.findById("wnd[0]/usr/ctxtS_DATA-LOW").Text = data
    session_monitor.findById("wnd[0]/usr/ctxtS_DATA-HIGH").Text = data
    session_monitor.findById("wnd[0]/usr/ctxtS_HORA-LOW").Text = hora_inicio
    session_monitor.findById("wnd[0]/usr/ctxtS_HORA-HIGH").Text = hora_fim
    session_monitor.findById("wnd[0]/usr/txtS_ORDEM-LOW").Type

In [10]:
# fazer interação ZSBMM204?
import sap
connection = sap.listar_conexoes()
session = connection.Children(1)
rodape = session.findById("wnd[0]/sbar").Text # Rodapé
if "ZSBMM204" in rodape:
    monitor204()

In [15]:
import sap
connection = sap.listar_conexoes()
session = connection.Children(3)


In [16]:
import pywintypes
try:
   monitor = session.findById("wnd[0]/usr/cntlCO_GRID/shellcont/shell")
   n_linhas = monitor.RowCount
   lista_monitor = []
   for i in range(n_linhas):
      mensagem = monitor.GetCellValue(i, "MENSAGEM")
      lista_monitor.append(mensagem)
except pywintypes.com_error:
    print("Erro.")

In [22]:
# Material bloqueado no monitor 204
import re
lista_monitor = str(lista_monitor)
lista_monitor = lista_monitor.lower()
if "bloqueados" in lista_monitor:
    padrao = r"material (\d+)"
    correspondencias = re.search(padrao, lista_monitor)
    if correspondencias:
        # Group 1 retira string 'material'
        codigo_material = correspondencias.group(1)
        print(codigo_material)

50001070


In [25]:
import sap
connection = sap.listar_conexoes()
session = connection.Children(1)
rodape = session.findById("wnd[0]/sbar").Text # Rodapé
salvo = "Ajustes de valoração salvos com sucesso."
if salvo == rodape:
    print(True)

True
